The notebook is used for convert json file from segmentation from Zm_StomataSeg to the PIXIMI Jason format.
The code is under the great help from Nodar  

In [27]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage.io as skio
import uuid
import os

In [4]:
def rle_encode(mask_arr, as_str=True):
  decoded = mask_arr.flatten()

  high_val = 1 # can be set to 2 ** bit_depth - 1 
  last_element = decoded[0]
  last_sequence_size = 1
  encoded = []
  
  if decoded[0] == high_val:
    encoded.append(0)

  for i in range(1, len(decoded)):
    if last_element != decoded[i]:
      encoded.append(last_sequence_size)
      last_element = decoded[i]
      last_sequence_size = 1
    else:
      last_sequence_size += 1
  
  encoded.append(last_sequence_size)
  
  if as_str:
    return ' '.join(str(x) for x in encoded)
  else:
    return encoded

In [5]:
def crop_mask(mask, bbox):
  mask = np.array(mask, dtype=np.uint8).reshape((len(mask), len(mask[0])))
  return mask[bbox[1]:bbox[3], bbox[0]:bbox[2]]

In [6]:
uuid4 = lambda: str(uuid.uuid4())

In [26]:
for file in Jason_full_path:  # Assuming Jason_full_path is a list of file paths
    filename, file_extension = os.path.splitext(file)
    if file_extension.lower() != '.json':
        print(f"Skipping file {file}. Not a JSON file.")
        continue

    print(filename)

    try:
        with open(file, 'r') as json_file:
            jj = json.load(json_file)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {file}: {e}")
        continue
    
    genotype = filename.split("_")[1]
    print(genotype)
    cat_map = {
    1: {'id': uuid4(), 'color': '#0000FF', 'name': 'A', 'visible': True},
    2: {'id': uuid4(), 'color': '#00FF00', 'name': 'B', 'visible': True},
    }
 
    im_map = {
        filename: {
            'id': uuid4(),
            'name': filename
        },
        genotype: {
            'id': uuid4(),
            'name': genotype
        }
    }
    print(im_map[filename]['id'])
   

    bbox_nrml = lambda bbox: list(map(lambda c: int(c), [bbox[0], bbox[1], bbox[2]-bbox[0], bbox[3]-bbox[1]]))
    bbox_nrml_coords = lambda bbox: list(map(lambda c: int(c), [bbox[0], bbox[1], bbox[2], bbox[3]]))

    piximi_annotations = []
    for ob in jj['objects']:
        bbox = bbox_nrml_coords(ob['bbox'])
        mask_arr = crop_mask(ob['mask'], bbox)

        piximi_annotations.append({
            'categoryId': cat_map[ob['class_name']]['id'],
            'imageId': im_map[filename]['id'],
            'genotype': im_map[genotype]['id'],
            'id': uuid4(),
            'plane': 0,
            'boundingBox': bbox,
            'mask': rle_encode(mask_arr)
            })
        
        piximi_json = {
            'categories': list(cat_map.values()),
            'images': list(im_map.values()),
            'annotations': piximi_annotations
        }
        with open('project_file.json', 'w') as f:
            f.write(json.dumps(piximi_json))


./Oscar_sample/Jason_file/Week3_CML228_1
sample/Jason
9d0708af-c689-4b30-9c8c-3edbed0d7b07
./Oscar_sample/Jason_file/Week1_CML103_3
sample/Jason
46f1d9ce-3933-4e49-8287-ffc56096ce6c
./Oscar_sample/Jason_file/Week2_Ki3_3
sample/Jason
decf80da-fcb9-46ea-9177-299cb25a84d1
./Oscar_sample/Jason_file/Week1_B73_1
sample/Jason
26255f6b-9d7c-4e01-87e5-b6a19e8f821a
./Oscar_sample/Jason_file/Week3_P39Gb_3
sample/Jason
1b0e523c-a7a9-469d-be4f-1245a32f47fd
./Oscar_sample/Jason_file/Week6_CML103_2
sample/Jason
c25881d2-3c6c-4bc5-9566-8999431410d5
./Oscar_sample/Jason_file/Week7_CML228_1
sample/Jason
87a69b9c-6ddf-4068-84df-2babecb76cd6
./Oscar_sample/Jason_file/Week2_CML103_2
sample/Jason
bea04c5e-feb7-4978-8e2b-a85a800c46b2
./Oscar_sample/Jason_file/Week3_NC358_5
sample/Jason
627f1e7e-55d2-490d-af7e-693b3a07aa7b
./Oscar_sample/Jason_file/Week6_B73_6
sample/Jason
75b5cfef-6e05-4900-900a-1ff66f1d8817
./Oscar_sample/Jason_file/Week1_CML228_3
sample/Jason
4a237d56-c20b-44ec-8ff7-1671c15b3f86
./Oscar_sa

In [66]:
piximi_json = {
  'categories': list(cat_map.values()),
  'images': list(im_map.values()),
  'annotations': piximi_annotations
}

In [67]:
with open('project_file.json', 'w') as f:
  f.write(json.dumps(piximi_json))